In [14]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
#Failed to retrieve MathJax from 'https://cdn.mathjax.org/mathjax/latest/MathJax.js'

In [53]:
a1=pd.read_csv('ctrlPeakStats.bed',delimiter='\t')
a1=a1.loc[0:9]
a1['Condition']='control'

a2=pd.read_csv('erlPeakStats.bed',delimiter='\t')
a2=a1.loc[0:9]
a2['Condition']='erl'

a3=pd.read_csv('upInCtrlPeakStats.bed',delimiter='\t')
a3=a1.loc[0:9]
a3['Condition']='upInCtrl'

a4=pd.read_csv('upInErlpeakStats.bed',delimiter='\t')
a4=a1.loc[0:9]
a4['Condition']='upInErl'

a=pd.concat([a1,a2,a3,a4])

#a[(a.Condition)=='control']
a

,Annotation,Number of peaks,Total size (bp),Log2 Enrichment,Condition
0,3UTR,422.0,21894917,0.226,control
1,miRNA,1.0,48690,0.317,control
2,ncRNA,201.0,5103632,1.257,control
3,TTS,758.0,27937354,0.719,control
4,pseudo,32.0,1828155,0.087,control
5,Exon,1012.0,34273806,0.841,control
6,Intron,18732.0,1190082929,-0.067,control
7,Intergenic,18777.0,1718624567,-0.593,control
8,Promoter,9497.0,31172590,4.208,control
9,5UTR,568.0,2495051,3.788,control


In [33]:
#Clean up the variables in memory
%reset_selective -f a[1-4]


In [52]:
#Helpful site for indexing http://stackoverflow.com/questions/21800169/python-pandas-get-index-of-rows-which-column-matches-certain-value

rollingPeaks=[]
a['PerPeaks']=0; PerPeaks=[]
condn=set(a['Condition'])
for i in condn:
    tempPeak = a['Number of peaks'].loc[a['Condition']==i] #Get the right condition
    tempPeak = map(float,tempPeak) #Convert to float
    tempPeak = tempPeak/np.sum(tempPeak)
    rollingPeaks = np.append(rollingPeaks,tempPeak)

a['PerPeaks']=rollingPeaks
a.head()
a[a['Annotation']=='TTS']

,Annotation,Number of peaks,Total size (bp),Log2 Enrichment,Condition,PerPeaks
3,TTS,758.0,27937354,0.719,control,0.01516
3,TTS,758.0,27937354,0.719,erl,0.01516
3,TTS,758.0,27937354,0.719,upInCtrl,0.01516
3,TTS,758.0,27937354,0.719,upInErl,0.01516


array([0, 1, 2, 3])

In [39]:
#Some code from http://matplotlib.org/examples/pylab_examples/bar_stacked.html


idx = np.arange(len(condn))

N = 5
#menMeans   = (20, 35, 30, 35, 27)
#womenMeans = (25, 32, 34, 20, 25)
#menStd     = (2, 3, 4, 1, 2)
#womenStd   = (3, 5, 2, 3, 3)
ind = np.arange(len(condn))    # the x locations for the groups
width = 0.35       # the width of the bars: can also be len(x) sequence

p1 = plt.bar(idx, menMeans,   width, color='r', yerr=womenStd)
p2 = plt.bar(idx, womenMeans, width, color='y',
             bottom=menMeans, yerr=menStd)

plt.ylabel('Scores')
plt.title('Scores by group and gender')
plt.xticks(ind+width/2., ('G1', 'G2', 'G3', 'G4', 'G5') )
plt.yticks(np.arange(0,81,10))
plt.legend( (p1[0], p2[0]), ('Men', 'Women') )

plt.show()

array([0, 1, 2, 3])

In [ ]:
#Stacked bar plots

In [ ]:
1